Notebook Documentation
This notebook demonstrates building a question-answering system that can respond in multiple languages (specifically Swahili and English in this example) using a combination of Retrieval Augmented Generation (RAG) and an agent with web search capabilities.

This cell installs all the required Python libraries for the project, including:

langchain: The core library for building LLM applications.
faiss-cpu: A library for efficient similarity search and clustering of dense vectors, used here for the vector store.
transformers: Used for loading pre-trained models, specifically for the translation models and embeddings.
torch: The deep learning framework, required by transformers.
serpapi: (Originally intended for SerpAPI, later replaced by DuckDuckGo) A Python client for the SerpAPI search API.
google-genai: The Google Generative AI SDK.
python-dotenv: Used for loading environment variables (though later switched to Colab secrets).
langchain-community: Contains community-contributed LangChain components, including tools like SerpAPIWrapper and DuckDuckGoSearchRun.
langchain_google_genai: LangChain integration for Google's Generative AI models.
google-search-results: A dependency for SerpAPI.

In [ ]:
# Install necessary libraries
!pip install langchain faiss-cpu transformers torch serpapi google-genai python-dotenv langchain-community langchain_google_genai google-search-results



This cell loads the necessary API keys from Google Colab's Secrets Manager. It retrieves the SERPAPI_API_KEY (though no longer used directly after switching to DuckDuckGo) and GOOGLE_API_KEY. It then prints a masked version of the keys to confirm they were loaded.

In [ ]:
# Load environment variables
from dotenv import load_dotenv
import os
from google.colab import userdata

# load_dotenv() # No longer needed when using userdata

#SERPAPI_API_KEY = userdata.get("SERPAPI_API_KEY")
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")


#print(f"SerpAPI Key: {SERPAPI_API_KEY[:2]}..." if SERPAPI_API_KEY else "SerpAPI Key not found")
print(f"Gemini API Key: {GOOGLE_API_KEY[:2]}..." if GOOGLE_API_KEY else "Google API Key not found")

SerpAPI Key: c5...
Gemini API Key: AI...


This cell imports various classes and functions needed for building the RAG system and the agent:

TextLoader, RecursiveCharacterTextSplitter: For loading and splitting text documents (though dummy data is used later).

HuggingFaceEmbeddings: For creating vector embeddings of text using a Hugging Face model.

FAISS: For creating a vector store from the embeddings.

RetrievalQA: A LangChain chain for performing question answering over retrieved documents.
PromptTemplate: For defining the structure of prompts sent to the language model.

SerpAPIWrapper: (Originally imported, later replaced in the tools list) A LangChain tool for using SerpAPI for web search.

initialize_agent, Tool: For creating and configuring the LangChain agent.

google.generativeai: The Google Generative AI SDK, used directly for some operations.

MarianMTModel, MarianTokenizer: From transformers, used for loading and using Marian translation models.

torch: For managing tensors and device placement for the translation models.This cell imports various classes and functions needed for building the RAG system and the agent:



In [ ]:
# Imports for LangChain and other systems
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper # Corrected import
from langchain.agents import initialize_agent, Tool
import google.generativeai as genai
from transformers import MarianMTModel, MarianTokenizer
import torch
#from serpapi import GoogleSearch

This cell configures the Google Generative AI client using the GOOGLE_API_KEY loaded from secrets. This is necessary before making any calls to the Gemini API.

In [ ]:
# Configure Gemini API client
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("Warning: Google API key is missing.")




This cell loads the MarianMT model and tokenizer for translation from Chinese to English (Helsinki-NLP/opus-mt-zh-en). Note: This model name is incorrect for Swahili to English and should be Helsinki-NLP/opus-mt-sw-en (if available) or a different appropriate model. The code attempts to move the model to a GPU if available.

In [ ]:
# Setup translation pipeline placeholders

# Load translation model and tokenizer for Swahili to English (example)
mt_model_name_sw_en = "Helsinki-NLP/opus-mt-zh-en"
mt_tokenizer_sw_en = MarianTokenizer.from_pretrained(mt_model_name_sw_en)
mt_model_sw_en = MarianMTModel.from_pretrained(mt_model_name_sw_en).to("cuda" if torch.cuda.is_available() else "cpu")



/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


This cell defines a Python function translate_to_english that takes text as input, tokenizes it using the loaded Swahili-to-English tokenizer, generates the translated text using the corresponding model, and decodes the output.

In [ ]:
# Function to translate to English
def translate_to_english(text):
    inputs = mt_tokenizer_sw_en(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(mt_model_sw_en.device) for k, v in inputs.items()}
    translated = mt_model_sw_en.generate(**inputs)
    return mt_tokenizer_sw_en.decode(translated[0], skip_special_tokens=True)


This cell initializes placeholder variables for the English to Swahili translation model and tokenizer. They are set to None and will be loaded when the translate_from_english function is called for the first time.

In [ ]:
# Placeholder for English to Swahili (or other language), will init later
mt_model_en_sw = None
mt_tokenizer_en_sw = None


This cell defines the translate_from_english function. It checks if the English to target language (defaulting to Swahili) model and tokenizer are already loaded. If not, it loads the appropriate MarianMT model (Helsinki-NLP/opus-mt-en-{target_language}) and tokenizer from Hugging Face and moves the model to the appropriate device. It then performs the translation similar to the translate_to_english function.

In [ ]:
# Function to translate from English
def translate_from_english(text, target_language="sw"):
    global mt_model_en_sw, mt_tokenizer_en_sw
    if mt_model_en_sw is None or mt_tokenizer_en_sw is None:
        model_name = f"Helsinki-NLP/opus-mt-en-{target_language}" # Corrected model name
        mt_tokenizer_en_sw = MarianTokenizer.from_pretrained(model_name)
        mt_model_en_sw = MarianMTModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
    inputs = mt_tokenizer_en_sw(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(mt_model_en_sw.device) for k, v in inputs.items()}
    translated = mt_model_en_sw.generate(**inputs)
    return mt_tokenizer_en_sw.decode(translated[0], skip_special_tokens=True)

To make this notebook smaller and runnable in a typical environment,
we will simulate document loading with dummy data in the next cell.


This cell contains a Python list sample_docs with strings representing short pieces of text about farming practices. This list serves as the "documents" for the RAG system.

In [ ]:
# Simulate loading documents
sample_docs = [
    "Crop rotation is essential to maintain soil health and increase yield.",
    "Water management helps avoid drought stress in plants.",
    "Use organic fertilizers to enrich soil nutrients.",
    "Timely planting improves harvest success.",
    "Improving soil fertility through cover cropping and adding compost can significantly increase crop yields.",
    "Proper pest and disease management is crucial for maximizing harvest size.",
    "Selecting high-yielding and disease-resistant crop varieties is key to increasing productivity.",
    "Optimizing planting density and spacing can improve resource utilization and lead to higher yields.",
    "Effective weed control reduces competition for nutrients and water, boosting crop production."
]

This cell converts the strings in sample_docs into Document objects, which is the format LangChain works with. It then uses RecursiveCharacterTextSplitter to split these documents into smaller chunks of a specified size with overlap. This is done to ensure that the chunks are small enough to be processed by the embedding model and to maintain context between chunks.

In [ ]:
# Convert to Document objects
from langchain_core.documents import Document

documents = [Document(page_content=text) for text in sample_docs]

# Split documents into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

len(docs)  # number of chunks

4

This cell initializes a HuggingFaceEmbeddings model using "sentence-transformers/all-MiniLM-L6-v2" to create vector representations (embeddings) of the text chunks. It then uses FAISS.from_documents to create a FAISS vector store from the document chunks and their embeddings. This vector store allows for efficient similarity search.

In [ ]:
# Create embeddings and vectorstore using Faiss
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)


/tmp/ipython-input-684200172.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


This cell creates a retriever from the FAISS vector store. The retriever is configured to use "similarity" search and retrieve the top 3 most similar documents (search_kwargs={"k":3}) when queried.

In [ ]:
# Retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})



This cell defines the Language Model (LLM) to be used. It uses ChatGoogleGenerativeAI from langchain_google_genai to wrap the "gemini-2.0-flash" model. A temperature of 0.0 is set for deterministic output. The GOOGLE_API_KEY is passed for authentication.

In [ ]:
# Define Gemini LLM wrapper
from langchain_google_genai import ChatGoogleGenerativeAI

# Use ChatGoogleGenerativeAI as the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0, google_api_key=GOOGLE_API_KEY)

This cell defines the prompt template for the RAG chain. It instructs the LLM to act as a helpful farming assistant and use the provided context to answer the question in simple language. It then creates a RetrievalQA chain, connecting the LLM, the retriever, and the prompt. chain_type="stuff" means all retrieved documents will be stuffed into the prompt. return_source_documents=True is set to get the source documents along with the answer.

In [ ]:
# Define Prompt Template and RetrievalQA Chain

template = '''
You are a helpful farming assistant. Use the context below to answer the question.

Context: {context}

Question: {question}

Answer in simple clear language.
'''

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=retriever, chain_type="stuff", return_source_documents=True, chain_type_kwargs={"prompt": prompt}
)

In [ ]:
%pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.1 MB/s eta 0:00:00


This cell initializes the DuckDuckGoSearchRun tool from langchain_community.tools. This tool allows the agent to perform web searches using DuckDuckGo. A list of tools is created, containing this "Web Search" tool.

In [ ]:
# Setup DuckDuckGo Search Tool
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(name="Web Search", func=search.run, description="Use this for searching up-to-date info on the web."),
]

This cell initializes a LangChain agent. It uses the initialize_agent function with the defined tools and the LLM. agent="zero-shot-react-description" specifies the agent type, which uses the ReAct framework for reasoning and acting. verbose=True is set to show the agent's thought process during execution.

In [ ]:
# Initialize agent combining Gemini LLM and ducks
from langchain.agents import initialize_agent

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)



This cell defines the main answer_question function. It takes a user question and target language (defaulting to English, though intended for Swahili) as input.

It translates the question to English if the input language is not English.
It runs the qa_chain (RAG) with the English query.
It checks if source_documents were returned by the RAG.
If no source documents were found, it prints a message and invokes the agent_executor with the English query to perform a web search.
If source documents were found, it uses the RAG answer.
Finally, it translates the final English answer back to the user's original language if it was not English.

In [ ]:
def answer_question(user_question, user_language="sw"):
    if user_language != "sw":
        query_en = translate_to_english(user_question)
    else:
        query_en = user_question

    # Run RAG first
    rag_output = qa_chain.invoke({"query": query_en}) # Use invoke and pass query as a dictionary
    rag_answer = rag_output['result'] # Access the answer using the 'result' key
    source_documents = rag_output['source_documents'] # Get source documents

    # Trigger web search if no source documents were found by RAG
    if not source_documents:
        print("RAG did not find relevant documents, falling back to web search...")
        web_answer = agent_executor.invoke({"input": query_en})['output'] # Use agent_executor.invoke
        final_answer = web_answer
    else:
        final_answer = rag_answer

    if user_language != "sw":
        translated_answer = translate_from_english(final_answer, target_language=user_language)
        return translated_answer
    else:
        return final_answer

This cell provides an example of calling the answer_question function with a question in Swahili ("Jinsi ya kuongeza mazao?") and prints the translated answer.

In [ ]:
# Example

question_sw = "Jinsi ya kuongeza mazao?"  # Swahili for "How to increase crop yields?"
ans = answer_question(question_sw, user_language="sw")
print("Answer:", ans)


Answer: I'm working perfectly fine! I'm here to help you with your farming by reminding you about important things like using organic fertilizers, rotating crops, and planting on time.


This cell re-initializes the LangChain agent with an updated prompt template. The prompt is modified to guide the agent more explicitly on when and how to use the "Web Search" tool, particularly when the RAG context is insufficient. It also includes instructions to avoid putting square brackets around the tool name in the "Action" step and adds handle_parsing_errors=True to the AgentExecutor for better error handling.

In [ ]:
# Initialize the Agent with an improved prompt
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate # Added import

agent_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can, focusing on providing helpful farming information. You have access to the following tools:

{tools}

The RAG context from the provided documents is: {agent_scratchpad}

If the provided RAG context is empty or does not contain relevant information to answer the question, use the Web Search tool to find the necessary information. **When using Web Search due to insufficient RAG context, base your answer solely on the information found through the web search.** Formulate your search query to be specific and include keywords related to the user's farming question.

Use the following format:

Question: the input question you must answer
Thought: you should always think step by step about what you are going to do. Consider if the RAG context is sufficient or if a web search is needed.
Action: the action to take, should be one of [{tool_names}] (do not include square brackets around the tool name in the Action)
Action Input: the input to the action (e.g., the search query for Web Search)
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:""")

agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
# Example question for the agent
question = "What are the best ways to improve soil health?"
answer = answer_question(question)
print(answer)

To make your soil healthy, here's what you can do:

*   **Use organic fertilizers:** These feed the soil naturally.
*   **Rotate your crops:** Plant different crops in the same area each season.


In [ ]:
# Example in Swahili
question_sw = "Jinsi ya kuongeza mazao?"  # Swahili for "How to increase crop yields?"
ans = answer_question(question_sw, user_language="sw")
print("Answer:", ans)

Answer: Ili kuongeza mazao:

*   Panda kwa wakati unaofaa.
*   Hakikisha mimea inapata maji ya kutosha.
*   Tumia mzunguko wa mazao.


In [ ]:
# Example in Swahili
question_sw = "Unaweza kuongezaje mavuno ya mazao kwenye shamba lako??"  # Swahili for "How to increase crop yields?"
ans = answer_question(question_sw, user_language="sw")
print("Answer:", ans)

Answer: Ninafanya kazi vizuri kabisa! Nipo hapa kukusaidia na ukulima wenu kwa kuwakumbusha mambo muhimu kama vile kutumia mbolea za kikaboni, kuzungusha mazao, na kupanda kwa wakati.


In [ ]:
# Example in Swahili
question_sw = "Unaweza kudhibiti vipi wadudu waharibifu kwenye shamba langu?"  # Swahili for "How to increase crop yields?"
ans = answer_question(question_sw, user_language="sw")
print("Answer:", ans)

Answer: Samahani, sijapewa taarifa kuhusu udhibiti wa wadudu waharibifu. Nimepewa taarifa kuhusu mbolea ya asili, mzunguko wa mazao, na upandaji kwa wakati.


In [ ]:
# Test the agent directly with a question that requires web search
print("Testing agent with web search query...")
agent_response = agent_executor.invoke({"input": "What are the latest advancements in drought-resistant crops?"})
print("Agent Output:", agent_response['output'])

Testing agent with web search query...


> Entering new AgentExecutor chain...
The question asks about the latest advancements in drought-resistant crops. This is a rapidly evolving field, so I will use a web search to find the most up-to-date information.
Action: [Web Search]
Action Input: "latest advancements drought resistant crops"[Web Search] is not a valid tool, try one of [Web Search].The question asks about the latest advancements in drought-resistant crops. This is a rapidly evolving field, so I will use a web search to find the most up-to-date information.
Action: [Web Search]
Action Input: "latest advancements in drought-resistant crops"[Web Search] is not a valid tool, try one of [Web Search].The question asks about the latest advancements in drought-resistant crops. This is a rapidly evolving field, so I will use a web search to find the most up-to-date information.
Action: [Web Search]
Action Input: "recent innovations drought resistant crops"[Web Search] is not a valid t